In [1]:
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim 
import requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
from math  import sqrt
CLIENT_ID = '240Z25PI35PAI1MTEYTMV2XF1TGLDRQEP5CFN2NZGSR25RS5' # your Foursquare ID
CLIENT_SECRET = '0LK0ZVFSRPR5QFYTG2E1VFVRNBMBXDOQDZH0MKZQN0C5GPMI' # your Foursquare Secret
VERSION = '20200209'
LIMIT = 1
radius = 1000
intent = 'browse'

In [2]:
dfResultsHealth = pd.DataFrame(columns=['name','lat','lng','categories','postcode'])
dfResultsFastF = pd.DataFrame(columns=['name','lat','lng','categories','postcode'])

In [3]:
def Square(x,y,z):
    global dfResultsHealth
    global dfResultsFastF
    neighborhood_latitude = x[0]
    neighborhood_longitude = x[1]
    postcode = x[2]
    query = y
    sortByPopularity = 1
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&intent={}&limit={}&query={}&sortByPopularity={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        neighborhood_latitude, 
        neighborhood_longitude, 
        radius,
        intent,
        LIMIT,
        query,
        sortByPopularity)
    res = requests.get(url).json()
    venues = res['response']
    venues = res['response']['groups'][0]['items']
    a = []
    if venues != a:
        if z == 'Health':
            nearby_venues = json_normalize(venues)
            dfObj = pd.DataFrame(nearby_venues)
            dfObj = dfObj[['venue.name','venue.location.lat','venue.location.lng','venue.categories']]
            dfObj.rename({'venue.name': 'name','venue.location.lat':'lat','venue.location.lng':'lng','venue.categories':'categories'}, axis=1, inplace=True)
            dfObj['postcode'] = postcode
            dfResultsHealth = dfResultsHealth.append(dfObj, ignore_index=True)
        else:
            nearby_venues = json_normalize(venues)
            dfObj = pd.DataFrame(nearby_venues)
            dfObj = dfObj[['venue.name','venue.location.lat','venue.location.lng','venue.categories']]
            dfObj.rename({'venue.name': 'name','venue.location.lat':'lat','venue.location.lng':'lng','venue.categories':'categories'}, axis=1, inplace=True)
            dfObj['postcode'] = postcode
            dfResultsFastF = dfResultsFastF.append(dfObj, ignore_index=True)
        

In [4]:
def normalize(cat):
    b = cat[0]
    b = json_normalize(b)
    b = b['name']
    return b

In [5]:
calls_geo = pd.read_csv("http://cocl.us/Geospatial_data", header = 0)
geo = pd.DataFrame(calls_geo)
geo.rename({'Postal Code': 'Postcode'}, axis=1, inplace=True)

In [6]:
geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
Postcode     103 non-null object
Latitude     103 non-null float64
Longitude    103 non-null float64
dtypes: float64(2), object(1)
memory usage: 2.5+ KB


In [7]:
results = geo[['Latitude','Longitude','Postcode']].apply(lambda x: Square(x,"healthy food",'Health'),axis = 1)

In [8]:
dfResultsHealth.head()

,name,lat,lng,categories,postcode
0,Mr. Greek,43.799853,-79.198234,"[{'id': '4bf58dd8d48988d10e941735', 'name': 'G...",M1B
1,Food Basics,43.770013,-79.185540,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",M1E
2,M&M Food Market,43.750638,-79.276521,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",M1P
3,Food Basics,43.746127,-79.288424,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",M1R
4,Bestco Food Market 鴻華超級市場,43.796514,-79.270790,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",M1S


In [9]:
dfResultsHealth['cat'] = dfResultsHealth[['categories','lat']].apply(lambda x:normalize(x),axis = 1)
dfResultsHealth.head()

,name,lat,lng,categories,postcode,cat
0,Mr. Greek,43.799853,-79.198234,"[{'id': '4bf58dd8d48988d10e941735', 'name': 'G...",M1B,Greek Restaurant
1,Food Basics,43.770013,-79.185540,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",M1E,Supermarket
2,M&M Food Market,43.750638,-79.276521,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",M1P,Grocery Store
3,Food Basics,43.746127,-79.288424,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",M1R,Supermarket
4,Bestco Food Market 鴻華超級市場,43.796514,-79.270790,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",M1S,Supermarket


In [10]:
dfResultsHealth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 6 columns):
name          60 non-null object
lat           60 non-null float64
lng           60 non-null float64
categories    60 non-null object
postcode      60 non-null object
cat           60 non-null object
dtypes: float64(2), object(4)
memory usage: 2.9+ KB


In [11]:
results2 = geo[['Latitude','Longitude','Postcode']].apply(lambda x: Square(x,"food truck,Fried Chicken,Fast Food Restaurant","FastF"),axis = 1)

In [12]:
dfResultsFastF.head()

,name,lat,lng,categories,postcode
0,Food Basics,43.770013,-79.185540,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",M1E
1,Natural Food Depot,43.745489,-79.293375,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",M1R
2,Bestco Food Market 鴻華超級市場,43.796514,-79.270790,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",M1S
3,Food Depot Supermarket 泰錦超級市場,43.776155,-79.307727,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",M1T
4,Harden Fast Food Ltd.,43.813820,-79.290700,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",M1V


In [13]:
dfResultsFastF['cat'] = dfResultsFastF[['categories','lat']].apply(lambda x:normalize(x),axis = 1)
dfResultsFastF.head()

,name,lat,lng,categories,postcode,cat
0,Food Basics,43.770013,-79.185540,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",M1E,Supermarket
1,Natural Food Depot,43.745489,-79.293375,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",M1R,Grocery Store
2,Bestco Food Market 鴻華超級市場,43.796514,-79.270790,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",M1S,Supermarket
3,Food Depot Supermarket 泰錦超級市場,43.776155,-79.307727,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",M1T,Grocery Store
4,Harden Fast Food Ltd.,43.813820,-79.290700,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",M1V,Fast Food Restaurant


In [14]:
dfResultsFastF.drop('categories', axis = 1,inplace=True)
dfResultsHealth.drop('categories', axis = 1,inplace=True)

In [15]:
dfResults = dfResultsHealth.merge(dfResultsFastF, how='left', left_on='postcode', right_on='postcode',
          suffixes=('_health', '_fastf'))

In [16]:
dfResults.head()

dfResults.fillna(value=0,inplace=True)

In [17]:
def distance(x):
    lat_health = x[0]
    lng_health = x[1]
    lat_fastf = x[2]
    lng_fastf = x[3]
    dlat = (x[0]-x[2])**2
    dlng = (x[1]-x[3])**2
    dh = dlat + dlng
    distance = sqrt(dh)
    return distance


dfResults['distance'] = dfResults[['lat_health','lng_health','lat_fastf','lng_fastf']].apply(lambda x: distance(x),axis = 1)


dfResults.head()

,name_health,lat_health,lng_health,postcode,cat_health,name_fastf,lat_fastf,lng_fastf,cat_fastf,distance
0,Mr. Greek,43.799853,-79.198234,M1B,Greek Restaurant,0,0.000000,0.000000,0,90.502969
1,Food Basics,43.770013,-79.185540,M1E,Supermarket,Food Basics,43.770013,-79.185540,Supermarket,0.000000
2,M&M Food Market,43.750638,-79.276521,M1P,Grocery Store,0,0.000000,0.000000,0,90.547695
3,Food Basics,43.746127,-79.288424,M1R,Supermarket,Natural Food Depot,43.745489,-79.293375,Grocery Store,0.004992
4,Bestco Food Market 鴻華超級市場,43.796514,-79.270790,M1S,Supermarket,Bestco Food Market 鴻華超級市場,43.796514,-79.270790,Supermarket,0.000000


In [18]:
dfResults.sort_values(by='distance', ascending=False, inplace=True)
dfResults.drop_duplicates('postcode', inplace=True)

In [19]:
dfResults.head(10)

,name_health,lat_health,lng_health,postcode,cat_health,name_fastf,lat_fastf,lng_fastf,cat_fastf,distance
48,Lucky Dragon Chinese Food,43.719463,-79.480755,M6L,Chinese Restaurant,0,0.0,0.0,0,90.711531
57,Artisano Bakery Café,43.631006,-79.518172,M8Z,Bakery,0,0.0,0.0,0,90.701733
49,Welcome Food Mart,43.669701,-79.483491,M6N,Grocery Store,0,0.0,0.0,0,90.689957
10,Food Basics,43.786720,-79.416991,M2M,Supermarket,0,0.0,0.0,0,90.688121
41,Lawrence Square,43.716164,-79.447026,M6B,Shopping Mall,0,0.0,0.0,0,90.680389
43,The Healthy Abode Inc. - Environmental Assessment,43.695864,-79.457912,M6E,Construction & Landscaping,0,0.0,0.0,0,90.680143
52,Carload Food Market,43.650150,-79.480454,M6S,Grocery Store,0,0.0,0.0,0,90.677881
50,Annette Food Market,43.662457,-79.470979,M6P,Italian Restaurant,0,0.0,0.0,0,90.675502
56,Ontario Food Market,43.601103,-79.500311,M8V,Grocery Store,0,0.0,0.0,0,90.671691
34,Bruno's Fine Food,43.736642,-79.419870,M5M,Butcher,0,0.0,0.0,0,90.666474
